In [1]:
from lib.ekyn import *
from lib.env import *
from sage.utils import *
from sage.models import *
from sklearn.metrics import ConfusionMatrixDisplay,f1_score
import pandas as pd
import os

states = {}

for experiments in [EXPERIMENTS_PATH,BEAST_EXPERIMENTS_PATH,BEAUTY_EXPERIMENTS_PATH]:
    for experiment in os.listdir(f'{experiments}'):
        if experiment == '.Trash-1000':
            continue
        if not os.path.exists(f'{experiments}/{experiment}/state.pt'):
            continue
        state = torch.load(f'{experiments}/{experiment}/state.pt',map_location='cpu',weights_only=False)
        states[experiment] = state

df = pd.DataFrame([states[experiment] for experiment in states])
pd.set_option('display.max_rows', 500)
df = df.sort_values(by='start_time',ascending=False)
df = df.reset_index(drop=True)

# df = df.iloc[[0,2]]
# df = df.iloc[:10]
info_df = df[['start_time','best_dev_loss','dropout','lr','wd','batch_size','norm','widthi','sequence_length','hidden_size','num_layers','frozen_encoder']]

display(info_df.sort_values(by='best_dev_loss'))

,start_time,best_dev_loss,dropout,lr,wd,batch_size,norm,widthi,sequence_length,hidden_size,num_layers,frozen_encoder
3,2024_28_08_14_14_58,0.172059,0.1,0.0003,0.01,512,NaN,NaN,7.0,32.0,1.0,True
1,2024_28_08_16_11_23,0.172120,0.1,0.0003,0.01,512,NaN,NaN,7.0,64.0,1.0,True
2,2024_28_08_14_42_32,0.172170,0.1,0.0003,0.01,512,NaN,NaN,9.0,32.0,1.0,True
4,2024_28_08_13_52_38,0.175536,0.1,0.0003,0.01,512,NaN,NaN,5.0,32.0,1.0,True
5,2024_28_08_13_50_33,0.186064,0.1,0.0003,0.01,512,NaN,NaN,3.0,32.0,1.0,True
0,2024_28_08_16_25_46,0.189282,0.1,0.0003,0.01,512,batch,NaN,7.0,32.0,1.0,False
15,2024_27_08_20_33_14,0.193337,0.1,0.0003,0.01,512,batch,NaN,3.0,4.0,1.0,True
7,2024_28_08_00_41_00,0.230921,0.4,0.0003,0.01,512,layer,[64],NaN,NaN,NaN,NaN
11,2024_27_08_21_38_46,0.231512,0.2,0.0003,0.01,512,layer,[64],NaN,NaN,NaN,NaN
18,2024_27_08_18_50_05,0.231558,0.1,0.0003,0.01,512,layer,[64],NaN,NaN,NaN,NaN


In [3]:
plot_loss_curves(df.iloc[:5],moving_window_length=1,lstm=True);

/home/andrew/sleep/sleep_ml/sage/utils.py:151: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  plot_df['best_dev_loss'] = plot_df['best_dev_loss'].astype(float)
/home/andrew/sleep/sleep_ml/sage/utils.py:152: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  plot_df['best_dev_f1'] = plot_df['best_dev_f1'].astype(float)
/home/andrew/sleep/sleep_ml/sage/utils.py:153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
from lib.ekyn import *
from lib.env import *
import copy
from sage.utils import *

MODEL_ID = f'2024_28_08_14_14_58'

state = torch.load(f'{BEAST_EXPERIMENTS_PATH}/{MODEL_ID}/state.pt',map_location='cpu',weights_only=False)
model = copy.deepcopy(state['model'])
model.load_state_dict(state['best_model_wts'])

trainloader,testloader = get_sequenced_dataloaders(
    batch_size=state['batch_size'],
    sequence_length=state['sequence_length'],
    )
loss,y_true,y_pred = evaluate(dataloader=testloader,model=model,criterion=state['criterion'],device='cuda')

ConfusionMatrixDisplay.from_predictions(y_true,y_pred,normalize='true',colorbar=False)
plt.title(f'f1 : {f1_score(y_true,y_pred,average="macro"):.3f}')
plt.savefig('dumbledore_a11.jpg')